In [ ]:
!wget --header="Host: datahack-prod.s3.amazonaws.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://datahack.analyticsvidhya.com/contest/practice-problem-age-detection/" "https://datahack-prod.s3.amazonaws.com/train_zip/train_DETg9GD.zip" -c -O 'train_DETg9GD.zip'

--2020-08-21 05:48:49--  https://datahack-prod.s3.amazonaws.com/train_zip/train_DETg9GD.zip
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.66.32
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.66.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50160162 (48M) [application/zip]
Saving to: ‘train_DETg9GD.zip’

train_DETg9GD.zip   100%[===================>]  47.84M  17.2MB/s    in 2.8s    

2020-08-21 05:48:52 (17.2 MB/s) - ‘train_DETg9GD.zip’ saved [50160162/50160162]



In [ ]:
!wget --header="Host: datahack-prod.s3.amazonaws.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://datahack.analyticsvidhya.com/contest/practice-problem-age-detection/" "https://datahack-prod.s3.amazonaws.com/test_zip/test_Bh8pGW3.zip" -c -O 'test_Bh8pGW3.zip'

--2020-08-21 05:48:57--  https://datahack-prod.s3.amazonaws.com/test_zip/test_Bh8pGW3.zip
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.62.84
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.62.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17007053 (16M) [application/zip]
Saving to: ‘test_Bh8pGW3.zip’

test_Bh8pGW3.zip    100%[===================>]  16.22M  12.5MB/s    in 1.3s    

2020-08-21 05:48:59 (12.5 MB/s) - ‘test_Bh8pGW3.zip’ saved [17007053/17007053]



In [ ]:
!unzip test_Bh8pGW3.zip
!unzip train_DETg9GD.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train/4047.jpg          
  inflating: Train/4049.jpg          
  inflating: Train/405.jpg           
  inflating: Train/4051.jpg          
  inflating: Train/4052.jpg          
  inflating: Train/4053.jpg          
  inflating: Train/4055.jpg          
  inflating: Train/4056.jpg          
  inflating: Train/4057.jpg          
  inflating: Train/4058.jpg          
  inflating: Train/4059.jpg          
  inflating: Train/406.jpg           
  inflating: Train/4060.jpg          
  inflating: Train/4061.jpg          
  inflating: Train/4062.jpg          
  inflating: Train/4065.jpg          
  inflating: Train/4066.jpg          
  inflating: Train/4068.jpg          
  inflating: Train/4069.jpg          
  inflating: Train/407.jpg           
  inflating: Train/4070.jpg          
  inflating: Train/4072.jpg          
  inflating: Train/4073.jpg          
  inflating: Train/4076.jpg          
  inflating: Train/4078.jpg          

In [ ]:
import pandas as pd
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
train

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE
...,...,...
19901,2482.jpg,MIDDLE
19902,20085.jpg,YOUNG
19903,19663.jpg,MIDDLE
19904,10132.jpg,MIDDLE


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen=ImageDataGenerator(rescale=1/255,validation_split=0.20)

In [ ]:
train_generator=datagen.flow_from_dataframe(dataframe=train,directory='Train',
                                            x_col="ID",y_col="Class",subset="training",batch_size=32,
                                            shuffle=True,class_mode="categorical",target_size=(100,100))

Found 15925 validated image filenames belonging to 3 classes.


In [ ]:
test_generator=datagen.flow_from_dataframe(dataframe=train,directory='Train',
                                            x_col="ID",y_col="Class",subset='validation',batch_size=32,
                                            shuffle=True,class_mode="categorical",target_size=(100,100))

Found 3981 validated image filenames belonging to 3 classes.


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Model

In [18]:
Rsnt_model = ResNet50(weights=None, include_top=False, input_shape=(100, 100, 3))

In [19]:
av1 = GlobalAveragePooling2D()(Rsnt_model.output)
fc1 = Dense(256, activation='relu')(av1)
drp1=Dropout(0.35)(fc1)
fc2 = Dense(128, activation='relu')(drp1)
drp2=Dropout(0.4)(fc2)
bat_norm=BatchNormalization()(drp2)
fc3 = Dense(68, activation='relu')(bat_norm)
drp3=Dropout(0.25)(fc3)
fc4 = Dense(34, activation='relu')(drp3)
out = Dense(3, activation='softmax')(fc3)
tl_model = Model(inputs=Rsnt_model.input,outputs=out)
tl_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [22]:
# compiling the model
tl_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# fitting the model
history = tl_model.fit_generator(train_generator,
                         steps_per_epoch = len(train_generator),
                         epochs = 15,
                         validation_data = test_generator,
                         validation_steps = len(test_generator),
                         verbose =1)

Epoch 1/15
498/498 [==============================] - 124s 248ms/step - loss: 0.3937 - accuracy: 0.8482 - val_loss: 0.6313 - val_accuracy: 0.7383
Epoch 2/15
498/498 [==============================] - 122s 245ms/step - loss: 0.3570 - accuracy: 0.8692 - val_loss: 0.5979 - val_accuracy: 0.7639
Epoch 3/15
498/498 [==============================] - 122s 246ms/step - loss: 0.3504 - accuracy: 0.8701 - val_loss: 0.5915 - val_accuracy: 0.7742
Epoch 4/15
498/498 [==============================] - 122s 245ms/step - loss: 0.3111 - accuracy: 0.8848 - val_loss: 0.5770 - val_accuracy: 0.7845
Epoch 5/15
498/498 [==============================] - 122s 245ms/step - loss: 0.2763 - accuracy: 0.9005 - val_loss: 0.5969 - val_accuracy: 0.7918
Epoch 6/15
498/498 [==============================] - 122s 244ms/step - loss: 0.2568 - accuracy: 0.9086 - val_loss: 0.6234 - val_accuracy: 0.7759
Epoch 7/15
498/498 [==============================] - 122s 245ms/step - loss: 0.2447 - accuracy: 0.9147 - val_loss: 0.6852 -